In [106]:
from pulp import *
from pandas import *
from numpy import *

# Creates the problem

prob = LpProblem("Products_Distribution_Problem", LpMinimize)

# Defining the warehouses, contracts, supply and demands

warehouses = ["A", "B", "C", "D", "E"]

supply = {"A": 175, "B": 175, "C": 175, "D": 175, "E": 175}

min_delivery = {"A": 25, "B": 30, "C": 30, "D": 35, "E": 25}

min_suppliers = {"1": 3, "2": 2, "3": 3, "4": 2, "5": 3, "6": 2, "7": 2, "8": 3, "9": 3, "10": 3}

contracts = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]

demands = {"1": 100, "2": 65, "3": 100, "4": 70, "5": 120, "6": 60, "7": 75, "8": 100, "9": 95, "10": 85}

# Defining the costs of transportation

costs = [
  [10, 15, 10, 15, 20, 20, 20, 40, 10, 30],
  [30, 15, 10, 20, 10, 20, 20, 30, 20, 30],
  [20, 10, 5, 15, 10, 15, 15, 10, 5, 5],
  [40, 25, 15, 20, 10, 30, 30, 10, 15, 10],
  [30, 30, 25, 10, 5, 35, 35, 15, 5, 10],
]

costs = makeDict([warehouses, contracts], costs, 0)

# Creating the tuples that contains all the arcs of possibilities

arcs = [(x, y) for x in warehouses for y in contracts]

# Creating the variables of the problem and 

vars = LpVariable.dicts("arc", (warehouses, contracts), 0, None, LpInteger)

is_warehouse_used = LpVariable.dicts("warehouses_used", (warehouses, contracts), 0, 1, LpInteger)

# Adding the objective function to the problem

prob += (
  lpSum([vars[x][y] * costs[x][y] for (x, y) in arcs]), "Total_Minimum_Cost",
)

# Adding the supply maximum restrictions of the problem

for x in warehouses:
  prob += (
    lpSum([vars[x][y] for y in contracts]) <= supply[x],
          "Max_Sum_of_Products_out_of_Warehouse_%s" % x,
  )

# Adding the contracts minimum restrictions of the problem

for y in contracts:
  prob += (
  lpSum([vars[x][y] for x in warehouses]) == demands[y],
        "Sum_of_Products_into_Contract%s" % y,
  )


# Adding the minimum suppliers restriction

for y in contracts:
  for x in warehouses:
    prob += (vars[x][y] >= min_delivery[x]*is_warehouse_used[x][y])
    prob += (vars[x][y] <= supply[x]*is_warehouse_used[x][y])

for y in contracts:
  prob += (
  lpSum([is_warehouse_used[x][y] for x in warehouses]) >= min_suppliers[y]
  )

# The problem data is written to an .lp file

prob.writeLP("ProductsDistributionProblem.lp")

# The problem is solved using PuLP's choice of Solver

result = prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print("Total Cost of Transportation = ", value(prob.objective))

# Printing the solution table

warehouses_name = ["Seattle", "San Francisco", "Las Vegas", "Phoenix", "Denver"]

contracts_list = []
for y in contracts:
  temp_list = []
  for x in warehouses:
    temp_list.append(value(vars[x][y]))
  contracts_list.append(temp_list)

prices_list = []
for y in contracts:
  temp_cost = 0
  for x in warehouses:
    temp_cost += value(vars[x][y])*value(costs[x][y])
  prices_list.append(temp_cost)

quantity_table = DataFrame(data = zip(warehouses_name, contracts_list[0], contracts_list[1], contracts_list[2], contracts_list[3], contracts_list[4], contracts_list[5], contracts_list[6], contracts_list[7], contracts_list[8], contracts_list[9]),
                  columns = ["Filiais" ,"Washington", "Oregon", "California", "Idaho", "Nevada", "Montana", "Wyoming", "Arizona", "Utah", "Colorado"],
        )

prices_table = DataFrame(array([prices_list]),
                  columns = ["Washington", "Oregon", "California", "Idaho", "Nevada", "Montana", "Wyoming", "Arizona", "Utah", "Colorado"],
        )

Status: Optimal
arc_A_1 = 45
arc_A_10 = 25
arc_A_2 = 25
arc_A_3 = 0
arc_A_4 = 25
arc_A_5 = 0
arc_A_6 = 25
arc_A_7 = 0
arc_A_8 = 0
arc_A_9 = 25
arc_B_1 = 0
arc_B_10 = 0
arc_B_2 = 40
arc_B_3 = 30
arc_B_4 = 0
arc_B_5 = 30
arc_B_6 = 35
arc_B_7 = 40
arc_B_8 = 0
arc_B_9 = 0
arc_C_1 = 30
arc_C_10 = 0
arc_C_2 = 0
arc_C_3 = 35
arc_C_4 = 0
arc_C_5 = 0
arc_C_6 = 0
arc_C_7 = 35
arc_C_8 = 30
arc_C_9 = 45
arc_D_1 = 0
arc_D_10 = 35
arc_D_2 = 0
arc_D_3 = 35
arc_D_4 = 0
arc_D_5 = 60
arc_D_6 = 0
arc_D_7 = 0
arc_D_8 = 45
arc_D_9 = 0
arc_E_1 = 25
arc_E_10 = 25
arc_E_2 = 0
arc_E_3 = 0
arc_E_4 = 45
arc_E_5 = 30
arc_E_6 = 0
arc_E_7 = 0
arc_E_8 = 25
arc_E_9 = 25
warehouses_used_A_1 = 1
warehouses_used_A_10 = 1
warehouses_used_A_2 = 1
warehouses_used_A_3 = 0
warehouses_used_A_4 = 1
warehouses_used_A_5 = 0
warehouses_used_A_6 = 1
warehouses_used_A_7 = 0
warehouses_used_A_8 = 0
warehouses_used_A_9 = 1
warehouses_used_B_1 = 0
warehouses_used_B_10 = 0
warehouses_used_B_2 = 1
warehouses_used_B_3 = 1
warehouses_used

In [107]:
quantity_table

,Filiais,Washington,Oregon,California,Idaho,Nevada,Montana,Wyoming,Arizona,Utah,Colorado
0,Seattle,45,25,0,25,0,25,0,0,25,25
1,San Francisco,0,40,30,0,30,35,40,0,0,0
2,Las Vegas,30,0,35,0,0,0,35,30,45,0
3,Phoenix,0,0,35,0,60,0,0,45,0,35
4,Denver,25,0,0,45,30,0,0,25,25,25


In [108]:
prices_table

,Washington,Oregon,California,Idaho,Nevada,Montana,Wyoming,Arizona,Utah,Colorado
0,1800,975,1000,825,1050,1200,1325,1125,600,1350
